In [ ]:
SELECT CONCAT(CustomerName, WebsiteURL) as "Customer Name And Web Site"
FROM Sales.Customers;

--Comment
SELECT LEFT(CustomerName,1) "CustomerInitial", Count(CustomerID) as COUNT
FROM Sales.Customers
Group by LEFT(CustomerName,1)


In [ ]:
SELECT 
    DATEPART(ww, o.OrderDate) as "Week",
    DATEPART(dw, o.OrderDate) as "DayofWeek",
    COUNT(OrderID) as "NumberOfOrders"
FROM Sales.Orders o
GROUP BY DATEPART(ww, o.OrderDate),DATEPART(dw, o.OrderDate)
ORDER BY 1, 2

This first section is going to explore projection part of SQL query, 

projection shape column result - starting with just column to apply functions to the columns

In [ ]:
SELECT FullName, LogonName, EmailAddress
FROM Application.People


In [ ]:
SELECT FullName, LogonName, EmailAddress, 
    CHARINDEX('@', EmailAddress) as Position,
    LEFT(EmailAddress, CHARINDEX('@', EmailAddress) - 1) as Mailbox,
    SUBSTRING(EmailAddress, CHARINDEX('@', EmailAddress)+ 1, 100) as DomainName
FROM Application.People
WHERE EmailAddress IS NOT NULL

In [ ]:
--present a list of people with capitalized initial firstName followed by last name
SELECT UPPER(LEFT(FullName, 1)) + '. ' + SUBSTRING(FullName, CHARINDEX(' ', FullName) + 1, 100) as Name, 
IIF(IsSalesperson=1, 'SalesPerson', IIF(IsEmployee=1, 'Employee', 'Customer')) as Role
FROM Application.People
ORDER BY Role DESC, LTRIM(SUBSTRING(FullName, CHARINDEX(' ', FullName) + 1, 100)) ASC

In [ ]:
--Mini Challenge, using the people table, give me a count of salesperson vs employees vs non-employees
Select COUNT(IIF(IsSalesperson=1, 1, null)) as Sales,
    COUNT(IIF(IsEmployee=1, 1, null)) as Employee,
    COUNT(IIF(IsSalesperson=0 AND IsEmployee=0, 1, null)) as Others
    FROM Application.People

In [ ]:
SELECT TOP 1 (SELECT COUNT(IsSalesperson) FROM Application.People WHERE IsSalesperson = 1) AS Sales,
    (Select COUNT(IsEmployee) FROM Application.People WHERE IsEmployee = 1) AS Employees
FROM Application.People

In [ ]:
Select Count(o.OrderID) as OrderCount, DaysToDeliver
FROM
(SELECT OrderID, DATEDIFF(DAY, o.OrderDate, o.ExpectedDeliveryDate) as DaysToDeliver
FROM Sales.Orders o) o
GROUP BY DaysToDeliver

In [ ]:
SELECT *
FROM Application.People WHERE FullName LIKE 'A%'

In [ ]:
SELECT COUNT(OrderID)
FROM Sales.Orders
WHERE OrderDate BETWEEN '2014-01-01' AND '2014-12-31' --Same as WHERE DATEPART(yyyy, OrderDate) = '2014'

In [ ]:
SELECT TOP 5 * FROM Sales.Invoices

In [ ]:
SELECT * FROM
(SELECT LineTotal, NTILE(3) OVER (
    PARTITION BY OrderID
    ORDER BY LineTotal DESC) as NthTile
    FROM
    (SELECT *, Quantity * UnitPrice AS LineTotal FROM Sales.OrderLines)
AS source) as NthTileResult

In [ ]:
SELECT OrderID, SUM(Quantity * UnitPrice) AS OrderTotal,
COUNT(OrderLineID) as NumberOfItems-- can only contain 1 group by expression, 2 aggregate reuslt
FROM Sales.OrderLines
GROUP BY OrderID
HAVING SUM(Quantity*UnitPrice) > 10000 AND COUNT(OrderLineID) > 2

In [ ]:
-- each person end of the month total for each month -- limit data to year 2015
--Customer  Month   NumOfTransactions   TransactionTotal
--Bob       Jan         16                  8561
SELECT CustomerID, DATEPART(mm, TransactionDate) AS Month,
COUNT(CustomerTransactionID) AS NumberOfTransactions,
SUM(TransactionAmount) AS TransactionTotal
FROM Sales.CustomerTransactions
WHERE DATEPART(yyyy, TransactionDate) = '2015'
GROUP BY CustomerID, DATEPART(mm, TransactionDate)
ORDER BY 1, 2, 3

In [ ]:
--produce a report of customer and their categories
-- Contoso      Wholesaler
-- Fabrikam     Warehouse
--Sales.Customer    Sales.CustomerCategories
SELECT CustomerName, CustomerCategoryName 
FROM Sales.Customers cust 
JOIN Sales.CustomerCategories cat 
ON cat.CustomerCategoryID = cust.CustomerCategoryID

In [ ]:
SELECT c.CountryName, COUNT(s.StateProvinceID)
FROM Application.Countries c
JOIN Application.StateProvinces s
ON s.CountryID = c.CountryID
GROUP BY c.CountryName

In [1]:
--per state how many vendors and customers are there city
SELECT s.StateProvinceName, COUNT(sup.SupplierID) AS NumOfSuppliers, COUNT(cust.CustomerID) as NumOfCustomers
FROM Application.StateProvinces s 
LEFT JOIN Application.Cities c ON s.StateProvinceID = c.StateProvinceID
LEFT JOIN Purchasing.Suppliers sup ON sup.PostalCityID = c.CityID
LEFT JOIN Sales.Customers cust ON c.CityID = cust.PostalCityID
GROUP BY s.StateProvinceName